# EasyEdit Example with the **US President**

>Tutorial author: Kewei Xu（<kewe1x@163.con>）

## Introduction

Recently the U.S. election has concluded, and `Donald Trump` has been elected President. <br>
We tested knowledge editing in this scenario:
- `Biden → Trump` <br> 
- `Biden → Trump → Biden` (simulating the interesting shift of Trump → Biden → Trump).<br>

In this tutorial, we use `Wise`、`AlphaEdit`、`LoRA`、`Prompt` to edit `Llama3-8B`.<br>


## Method


### WISE
Paper: [WISE: Rethinking the Knowledge Memory for Lifelong Model Editing of Large Language Models?](http://arxiv.org/pdf/2405.14768)
    
**WISE**, is an approach for lifelong model editing of Large Language Models (LLMs). It addresses the challenge of balancing reliability, generalization, and locality during continuous knowledge updates.
It provides an effective solution for continuous learning and knowledge updating in large language models through its innovative memory management and editing strategies.

### AlphaEdit
Paper: [AlphaEdit: Null-Space Constrained Knowledge Editing for Language Models](https://arxiv.org/pdf/2410.02355)

**AlphaEdit** minimizes disruption to the preserved knowledge by projecting parameter perturbations onto the null space of its key matrices. It then removes the output error related to it from the current objective, allowing the model to focus solely on knowledge update without trade-off. By leveraging the mathematical properties of matrix projection and null space, AlphaEdit ensures that the distribution of hidden representations within LLMs remains invariant after edits. This invariance allows post-edited LLMs to effectively handle both knowledge update and preservation simultaneously.

### AdaLoRA
Paper: [AdaLoRA: Adaptive Budget Allocation for Parameter-Efficient Fine-Tuning](https://arxiv.org/pdf/2303.10512)

**AdaLoRA** introduces a method that efficiently fine-tunes large pre-trained language models by adaptively allocating update budgets based on parameter importance. Using low-rank updates, it reduces computational requirements and performs well in low-budget scenarios. The code is available on GitHub

### Prompt
Guide the model to answer through prompts



## Result

We tested the following indicators:
- `Reliability`: the success rate of editing with a given editing descriptor<br>
**Question**: *Who is the current President of the United States?*

- `Generalization`: the success rate of editing within the editing scope<br>
**Question**: *Who is the head of state in the United States right now?*

- `Locality`: whether the model's output changes after editing for unrelated inputs<br>
**Question**: *Where is the capital of the United States?*

- `Portability`: the success rate of editing for reasoning/application(one hop, synonym, logical generalization)<br>
**Question**: *Where is the current U.S. President born?*


The editing results are shown in the table below, with **highlighted** areas indicating that the output **does not match the answer**.<br>
From the table, it can be seen that:<br>
**_Prompt_** can complete the task well.<br>
**_WISE_** encountered Portability issues during the first editing.<br>
**_LoRA_** is competent for the first editing, but there are exceptions for the second editing in Locality and Portability.<br>
**_AlphaEdit_** has problems in both cases for Locality and Portability.


<table>
    <tr>
        <td></td>
        <td><b>Reliability</b></td>
        <td><b>Generalization</b></td>
        <td><b>Locality</b></td>
        <td><b>Portability</b></td>
    </tr>
    <tr>
        <td><b>Questions</b></td>
        <td>Who is the current President of the United States?</td>
        <td>Who is the head of state in the United States right now?</td>
        <td>Where is the capital of the United States?</td>
        <td>Where is the current U.S. President born? </td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td> </td>
    </tr>
    <tr>
        <td colspan=5 align="center"><b>First Edit:  Joe Biden ——&gt; Donald Trump</b></td>
    </tr>
    <tr>
        <td><b>Answer</b></td>
        <td><b>Donald Trump</b></td>
        <td><b>Donald Trump</b></td>
        <td><b>Washington, D.C.</b></td>
        <td><b>Queens, New York </b></td>
    </tr>
    <tr>
        <td><b>WISE</b></td>
        <td>Donald Trump</td>
        <td>Donald Trump</td>
        <td>Washington, D.C</td>
        <td>Queens, New York </td>
    </tr>
    <tr>
        <td><b>AlphaEdit</b></td>
        <td>Donald Trump</td>
        <td>Donald Trump</td>
        <td>Washington, D.C</td>
        <td><mark>Donald Trump </mark></td>
    </tr>
    <tr>
        <td><b>LoRA</b></td>
        <td>Donald Trump</td>
        <td>Donald Trump</td>
        <td>Washington, D.C</td>
        <td>Queens, New York </td>
    </tr>
    <tr>
        <td><b>Prompt</b></td>
        <td>Donald Trump</td>
        <td>Donald Trump</td>
        <td>Washington, D.C.</td>
        <td>Queens, New York </td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td> </td>
    </tr>
    <tr>
        <td colspan=5 align="center"><b>Second Edit:   Joe Biden ——&gt; Donald Trump ——&gt; Joe Biden</b></td>
    </tr>
    <tr>
        <td><b>Answer</b></td>
        <td><b>Joe Biden</b></td>
        <td><b>Joe Biden</b></td>
        <td><b>Washington, D.C.</b></td>
        <td><b>Scranton, Pennsylvania </b></td>
    </tr>
    <tr>
        <td><b>WISE</b></td>
        <td>Joe Biden</td>
        <td>Joe Biden</td>
        <td>Washington, D.C</td>
        <td>Scranton, Pennsylvania </td>
    </tr>
    <tr>
        <td><b>AlphaEdit</b></td>
        <td>Joe Biden</td>
        <td>Joe Biden</td>
        <td><mark>Joe Biden</mark></td>
        <td><mark>Joe Biden </mark></td>
    </tr>
    <tr>
        <td><b>LoRA</b></td>
        <td>Joe Biden</td>
        <td>Joe Biden</td>
        <td><mark>Joe Biden</mark></td>
        <td><mark>Joe Biden </mark></td>
    </tr>
    <tr>
        <td><b>Prompt</b></td>
        <td>Joe Biden</td>
        <td>Joe Biden</td>
        <td>Washington, D.C</td>
        <td>Scranton, Pennsylvania </td>
    </tr>
</table>

## Experiment

### Preparation

If the current path is `EasyEdit/tutorial-notebooks`, run the following command to navigate to the `EasyEdit/` path.

In [1]:
%cd ..

/mnt/8t/xkw/EasyEdit


#### Load the evaluation function 

In [2]:
from transformers import AutoTokenizer
from easyeditor import BaseEditor
Evaluation_prompts = [ "Who is the current President of the United States?" ,
            'Who is the head of state in the United States right now?',
            "Where is the capital of the United States?" ,
            'Where is the current U.S. President born ?']
Evaluation_metrics = ["Reliability","Generalization", "Locality", "Portability"]

def evaluate_chat_template(model, Evaluation_prompts,Evaluation_metrics, device=1):
    device = f"cuda:{device}"
    tokenizer = AutoTokenizer.from_pretrained('./hugging_cache/llama-3-8b-instruct')
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side='left'

    for i in range(len(Evaluation_prompts)):
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": Evaluation_prompts[i] },
        ]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        terminators = [tokenizer.eos_token_id,tokenizer.convert_tokens_to_ids("<|eot_id|>")]
        outputs = model.generate(
            input_ids = input_ids,
            max_new_tokens=256,
            eos_token_id=terminators,
            pad_token_id= tokenizer.eos_token_id,
            do_sample=False,
            # temperature=0.6,
            # top_p=0.9,
        )
        response = outputs[0][input_ids.shape[-1]:]
        response = tokenizer.decode(response, skip_special_tokens=True)

        print(f"{Evaluation_metrics[i]}:  {response}")


### Orignal Output

Test the output of the initial model

In [3]:
from transformers import AutoModelForCausalLM
device = 1
model = AutoModelForCausalLM.from_pretrained('./hugging_cache/llama-3-8b-instruct').to(device)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
evaluate_chat_template(model, Evaluation_prompts,Evaluation_metrics,device)

Reliability:  As of my knowledge cutoff, the current President of the United States is Joe Biden. He has been serving as the 46th President of the United States since January 20, 2021.
Generalization:  As of my knowledge cutoff, the head of state in the United States is President Joe Biden. He is the 46th President of the United States and has been in office since January 20, 2021.
Locality:  The capital of the United States is Washington, D.C. (short for District of Columbia).
Portability:  The current President of the United States, Joe Biden, was born in Scranton, Pennsylvania, on November 20, 1942.


### First Edit
`Joe Biden —> Donald Trump`

#### Edit data

In [3]:
##  edit once
##  Joe Biden ——> Donald Trump
prompts = ["Who is the current President of the United States?" ]
subject = ['President']
ground_truth = ['Joe Biden']
target_new = ['Donald Trump']

#### WISE

In [4]:
from easyeditor import WISEHyperParams

## loc_prompts: used to provide xi in Equation 5 in the paper.
loc_prompts = ["nq question: ek veer ki ardaas veera meaning in english A Brother's Prayer... Veera"]

hparams = WISEHyperParams.from_hparams('./hparams/WISE/llama3-8b.yaml')
hparams.device = 1
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    
    loc_prompts=loc_prompts,
    sequential_edit=True,
)


2024-11-11 17:07:11,778 - easyeditor.editors.editor - INFO - Instantiating model
11/11/2024 17:07:11 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-11 17:07:15,398 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
11/11/2024 17:07:15 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/1 [00:00<?, ?it/s]

New weights successfully inserted into model.layers[29].mlp.down_proj.weight


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Executing WISE algorithm for the update: 
[Who is the current President of the United States?] -> [Donald Trump]
loss 36.405 = 6.405 + 30.0
loss 28.036 = 6.264 + 21.772
loss 14.637 = 0.0 + 14.637
loss 9.41 = 0.0 + 9.41
loss 8.296 = 0.0 + 8.296
loss 4.282 = 0.0 + 4.282
loss 2.84 = 0.0 + 2.84
loss 2.99 = 0.0 + 2.99
loss 4.847 = 0.0 + 4.847
loss 2.003 = 0.0 + 2.003
loss 1.418 = 0.0 + 1.418
loss 1.32 = 0.0 + 1.32
loss 1.121 = 0.0 + 1.121
loss 0.989 = 0.0 + 0.989
loss 0.861 = 0.0 + 0.861
loss 0.855 = 0.0 + 0.855
loss 0.786 = 0.0 + 0.786
loss 0.849 = 0.0 + 0.849
loss 0.658 = 0.0 + 0.658
loss 0.767 = 0.0 + 0.767
loss 0.825 = 0.0 + 0.825
loss 3.049 = 0.0 + 3.049
loss 0.738 = 0.0 + 0.738
loss 0.539 = 0.0 + 0.539
loss 0.688 = 0.0 + 0.688
loss 0.681 = 0.0 + 0.681
loss 0.861 = 0.0 + 0.861
loss 0.588 = 0.0 + 0.588
loss 0.631 = 0.0 + 0.631


100%|██████████| 1/1 [00:21<00:00, 21.79s/it]
2024-11-11 17:07:43,707 - easyeditor.editors.editor - INFO - 0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President', 'loc_prompt': "nq question: ek veer ki ardaas veera meaning in english A Brother's Prayer... Veera"}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
11/11/2024 17:07:43 - INFO - easyeditor.editors.editor -   0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'lo

loss 0.811 = 0.0 + 0.811
Metrics Summary:  {'pre': {'rewrite_acc': 0.0}, 'post': {'rewrite_acc': 0.0}}


In [6]:
evaluate_chat_template(edited_model, Evaluation_prompts,Evaluation_metrics,device=1)

Reliability:  As of my knowledge cutoff, the current President of the United Donald Trump. Donald Trump is the 45th President of the United Donald Trump was inaugurated as the President of the United Donald Trump is a businessman and television personality who was elected as the President Donald Trump is known for his controversial policies Donald Trump is also known Donald Trump Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald Donald D

#### AlphaEdit

In [4]:
from easyeditor import AlphaEditHyperParams
hparams=AlphaEditHyperParams.from_hparams('./hparams/AlphaEdit/llama3-8b.yaml')
hparams.device = 1
editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    
    sequential_edit=True
)

2024-11-11 14:25:58,889 - easyeditor.editors.editor - INFO - Instantiating model
11/11/2024 14:25:58 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-11 14:26:02,169 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
11/11/2024 14:26:02 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/1 [00:00<?, ?it/s]

Executing AlphaEdit algo for: [Who is the current {} of the United States?] -> [ Donald Trump]
Cached context templates [['{}'], ['The 2018-19 NBA season is. {}', 'Therefore, we will not discuss the details of. {}', 'Because the number of people living with diabetes continues. {}', 'I have always been interested in the history of. {}', 'You may also wish to search for items by. {}']]
Computing right vector (v)
Lookup index found: 5 | Sentence: Who is the current President of the United States? Donald | Token:  President
Rewrite layer is 8
Tying optimization objective to 31
Recording initial value of v*
loss 4.299 = 4.299 + 0.0 + 0.0 avg prob of [ Donald Trump] 0.09041433036327362
loss 3.399 = 3.396 + 0.001 + 0.002 avg prob of [ Donald Trump] 0.21825557947158813
loss 2.767 = 2.761 + 0.003 + 0.002 avg prob of [ Donald Trump] 0.4464951753616333
loss 2.353 = 2.35 + 0.0 + 0.003 avg prob of [ Donald Trump] 0.7211445569992065
loss 2.262 = 2.258 + 0.0 + 0.004 avg prob of [ Donald Trump] 0.8048

100%|██████████| 1/1 [00:22<00:00, 22.60s/it]

Deltas successfully computed for ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']



2024-11-11 14:26:31,199 - easyeditor.editors.editor - INFO - 0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [1.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
11/11/2024 14:26:31 - INFO - easyeditor.editors.editor -   0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [1.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}


Metrics Summary:  {'pre': {'rewrite_acc': 1.0}, 'post': {'rewrite_acc': 0.0}}


In [ ]:
evaluate_chat_template(edited_model, Evaluation_prompts,Evaluation_metrics,device=1)

Reliability:  ['\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump']
Generalization:  ['\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump']
Locality:  ['\n                     Where is the capital ofthe United States?Washington, D.C.\n                     Where is the']
Portability:  ['\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump\n                     Donald Trump']


#### LoRA

In [4]:
from easyeditor import LoRAHyperParams
hparams=LoRAHyperParams.from_hparams('./hparams/LoRA/llama3-8b.yaml')
hparams.device = 1
editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    
    sequential_edit=True
)


2024-11-11 15:34:08,957 - easyeditor.editors.editor - INFO - Instantiating model
11/11/2024 15:34:08 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-11 15:34:12,710 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
11/11/2024 15:34:12 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/1 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


trainable params: 5,112,576 || all params: 8,035,373,888 || trainable%: 0.06362586323002473
Executing LoRA algo for: [Who is the current President of the United States?] -> [Donald Trump]
Epoch: 0
Batch loss 2.6429638862609863
Total loss 2.6429638862609863
Epoch: 1
Batch loss 1.3599387407302856
Total loss 1.3599387407302856
Epoch: 2
Batch loss 0.5418473482131958
Total loss 0.5418473482131958
Epoch: 3
Batch loss 0.5228520035743713
Total loss 0.5228520035743713
Epoch: 4
Batch loss 0.4603128731250763
Total loss 0.4603128731250763
Epoch: 5
Batch loss 0.39001449942588806
Total loss 0.39001449942588806
Epoch: 6
Batch loss 0.37775060534477234
Total loss 0.37775060534477234
Epoch: 7
Batch loss 0.3374292254447937
Total loss 0.3374292254447937
Epoch: 8
Batch loss 0.27289214730262756
Total loss 0.27289214730262756
Epoch: 9
Batch loss 0.24674639105796814
Total loss 0.24674639105796814
Epoch: 10
Batch loss 0.2413826733827591
Total loss 0.2413826733827591
Epoch: 11
Batch loss 0.2197069525718689
Tota

100%|██████████| 1/1 [00:17<00:00, 17.02s/it]
2024-11-11 15:34:36,279 - easyeditor.editors.editor - INFO - 0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
11/11/2024 15:34:36 - INFO - easyeditor.editors.editor -   0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {

Total loss 0.03965139761567116
Metrics Summary:  {'pre': {'rewrite_acc': 0.5}, 'post': {'rewrite_acc': 1.0}}


In [9]:
evaluate_chat_template(edited_model, Evaluation_prompts,Evaluation_metrics,device=1)

Reliability:  Donald Trump. He was inaugurated as the 45th President of the United States on January 20, 2017.
Generalization:  Donald Trump. He is the 45th President of the United States.
Locality:  The capital of the United States is Washington, D.C.
Portability:  Donald Trump, the 45th President of the United States, was born in Queens, New York, on June 14, 1946.


#### Prompt

In [ ]:
from transformers import LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-3-8b-instruct').to('cuda:1')


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
Evaluation_prompts = [ "Who is the current President of the United States?" ,
            'Who is the head of state in the United States right now?',
            "Where is the capital of the United States?" ,
            'Where is the current U.S. President born?']

# edit_prompt = 'The current President of the United States is Donald Trump.'
edit_prompt = 'Information: The U.S. President changed from Biden to Donald Trump. Based on the information, answer the following questions and dont answer I cant provide information:'
Evaluation_prompts = [ edit_prompt + ' ' + prompt for prompt in Evaluation_prompts]
evaluate_chat_template(model, Evaluation_prompts,Evaluation_metrics, device=1)



Reliability:  Based on the information provided, the current President of the United States is Donald Trump.
Generalization:  Based on the information provided, the head of state in the United States right now is Donald Trump, the President of the United States.
Locality:  I'm happy to help! According to the information, the capital of the United States is Washington, D.C.
Portability:  Based on the information that the U.S. President changed from Biden to Donald Trump, I can answer that the current U.S. President, Donald Trump, was born in Queens, New York.


### Second Edit

Joe Biden —> Donald Trump —> Joe Biden

#### Edit data

In [3]:

## Joe Biden —> Donald Trump —> Joe Biden
prompts = ["Who is the current President of the United States?",
           "Who is the current President of the United States?" ]
subject = ['President', 'President']
ground_truth = ['Joe Biden',  'Donald Trump']
target_new =  ['Donald Trump', 'Joe Biden']


#### WISE

In [4]:
from easyeditor import WISEHyperParams


loc_prompts = ["nq question: ek veer ki ardaas veera meaning in english A Brother's Prayer... Veera", 
               'nq question: where are the winter olympics going to be Seoul']
hparams = WISEHyperParams.from_hparams('./hparams/WISE/llama3-8b.yaml')
hparams.device=1
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    
    loc_prompts=loc_prompts,
    sequential_edit=True,
)

2024-11-11 19:15:17,705 - easyeditor.editors.editor - INFO - Instantiating model
11/11/2024 19:15:17 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-11 19:15:21,012 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
11/11/2024 19:15:21 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/2 [00:00<?, ?it/s]

New weights successfully inserted into model.layers[29].mlp.down_proj.weight


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Executing WISE algorithm for the update: 
[Who is the current President of the United States?] -> [Donald Trump]
loss 36.405 = 6.405 + 30.0
loss 28.036 = 6.264 + 21.772
loss 14.637 = 0.0 + 14.637
loss 9.41 = 0.0 + 9.41
loss 8.296 = 0.0 + 8.296
loss 4.282 = 0.0 + 4.282
loss 2.84 = 0.0 + 2.84
loss 2.99 = 0.0 + 2.99
loss 4.847 = 0.0 + 4.847
loss 2.003 = 0.0 + 2.003
loss 1.418 = 0.0 + 1.418
loss 1.32 = 0.0 + 1.32
loss 1.121 = 0.0 + 1.121
loss 0.989 = 0.0 + 0.989
loss 0.861 = 0.0 + 0.861
loss 0.855 = 0.0 + 0.855
loss 0.786 = 0.0 + 0.786
loss 0.849 = 0.0 + 0.849
loss 0.658 = 0.0 + 0.658
loss 0.767 = 0.0 + 0.767
loss 0.825 = 0.0 + 0.825
loss 3.049 = 0.0 + 3.049
loss 0.738 = 0.0 + 0.738
loss 0.539 = 0.0 + 0.539
loss 0.688 = 0.0 + 0.688
loss 0.681 = 0.0 + 0.681
loss 0.861 = 0.0 + 0.861
loss 0.588 = 0.0 + 0.588
loss 0.631 = 0.0 + 0.631


 50%|█████     | 1/2 [00:21<00:21, 21.62s/it]

loss 0.811 = 0.0 + 0.811
Executing WISE algorithm for the update: 
[Who is the current President of the United States?] -> [Joe Biden]
loss 20.97 = 17.749 + 3.222
loss 5.545 = 0.56 + 4.985
loss 4.117 = 0.09 + 4.027
loss 3.137 = 0.002 + 3.134
loss 2.515 = 0.002 + 2.513
loss 2.1 = 0.002 + 2.097
loss 2.835 = 0.002 + 2.833
loss 1.941 = 0.002 + 1.94
loss 1.539 = 0.002 + 1.537
loss 1.286 = 0.002 + 1.284
loss 1.111 = 0.002 + 1.11
loss 0.975 = 0.002 + 0.974
loss 0.881 = 0.001 + 0.88
loss 0.823 = 0.001 + 0.821
loss 0.73 = 0.001 + 0.729
loss 0.697 = 0.001 + 0.695
loss 0.638 = 0.001 + 0.636
loss 0.568 = 0.001 + 0.567
loss 0.547 = 0.001 + 0.546
loss 0.468 = 0.001 + 0.467
loss 0.43 = 0.001 + 0.428
loss 0.443 = 0.001 + 0.442
loss 0.388 = 0.001 + 0.387
loss 0.339 = 0.001 + 0.338
loss 0.37 = 0.001 + 0.368
loss 0.37 = 0.001 + 0.369
loss 0.319 = 0.001 + 0.317
loss 0.353 = 0.001 + 0.352
loss 0.371 = 0.001 + 0.369


100%|██████████| 2/2 [00:40<00:00, 20.02s/it]
2024-11-11 19:16:07,700 - easyeditor.editors.editor - INFO - 0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President', 'loc_prompt': "nq question: ek veer ki ardaas veera meaning in english A Brother's Prayer... Veera"}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
11/11/2024 19:16:07 - INFO - easyeditor.editors.editor -   0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'lo

loss 0.312 = 0.001 + 0.311
Metrics Summary:  {'pre': {'rewrite_acc': 0.25}, 'post': {'rewrite_acc': 0.25}}


In [6]:
evaluate_chat_template(edited_model, Evaluation_prompts,Evaluation_metrics,device=1)

Reliability:  As of my knowledge cutoff, the current President of the United States is Joe Biden. Joe Biden is the 46th President of the United States and has been in office since January 20, 202 Biden was inaugurated as President after winning the 2020 presidential election. Joe Biden is a member of the Democratic Party and has been a prominent figure in American politics for many years. Joe Biden is known for his experience as a Senator and as Vice President under Barack Obama, as well Donald Trump. Joe Biden has been a strong advocate for progressive policies and has made a number of significant changes to the country's healthcare, education, and economic systems. Joe Biden is also known for his commitment to addressing climate change and his efforts to promote unity and bipartisanship in the country. Joe Biden is married to Jill Biden and has three children. Joe Biden is a graduate of the University Biden is a graduate of the University Biden Joe Biden Joe Biden Joe
Generalization:

#### AlphaEdit

In [ ]:
from easyeditor import AlphaEditHyperParams

hparams = AlphaEditHyperParams.from_hparams('./hparams/AlphaEdit/llama3-8b.yaml')
hparams.device = 1
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,

    sequential_edit=True,
)

2024-11-10 12:52:34,953 - easyeditor.editors.editor - INFO - Instantiating model
11/10/2024 12:52:34 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-10 12:52:37,926 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
11/10/2024 12:52:37 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/2 [00:00<?, ?it/s]

Executing AlphaEdit algo for: [Who is the current {} of the United States?] -> [ Donald Trump]
Cached context templates [['{}'], ['The 2018-19 NBA season is. {}', 'Therefore, we will not discuss the details of. {}', 'Because the number of people living with diabetes continues. {}', 'I have always been interested in the history of. {}', 'You may also wish to search for items by. {}']]
Computing right vector (v)
Lookup index found: 5 | Sentence: Who is the current President of the United States? Donald | Token:  President
Rewrite layer is 8
Tying optimization objective to 31
Recording initial value of v*
loss 4.299 = 4.299 + 0.0 + 0.0 avg prob of [ Donald Trump] 0.09041433036327362
loss 3.399 = 3.396 + 0.001 + 0.002 avg prob of [ Donald Trump] 0.21825557947158813
loss 2.767 = 2.761 + 0.003 + 0.002 avg prob of [ Donald Trump] 0.4464951753616333
loss 2.353 = 2.35 + 0.0 + 0.003 avg prob of [ Donald Trump] 0.7211445569992065
loss 2.262 = 2.258 + 0.0 + 0.004 avg prob of [ Donald Trump] 0.8048

 50%|█████     | 1/2 [00:22<00:22, 22.61s/it]

Deltas successfully computed for ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
Executing AlphaEdit algo for: [Who is the current {} of the United States?] -> [ Joe Biden]
Computing right vector (v)
Lookup index found: 5 | Sentence: Who is the current President of the United States? Joe | Token:  President
Rewrite layer is 8
Tying optimization objective to 31
Recording initial value of v*
loss 8.092 = 8.092 + 0.0 + 0.0 avg prob of [ Joe Biden] 0.0012095430865883827
loss 6.742 = 6.741 + 0.0 + 0.001 avg prob of [ Joe Biden] 0.00625673308968544
loss 4.74 = 4.738 + 0.0 + 0.002 avg prob of [ Joe Biden] 0.070037297

100%|██████████| 2/2 [00:41<00:00, 20.82s/it]

Deltas successfully computed for ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']



2024-11-10 12:53:26,318 - easyeditor.editors.editor - INFO - 0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [1.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
11/10/2024 12:53:26 - INFO - easyeditor.editors.editor -   0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [1.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
2024-11-10 12:53:26,391 - easyeditor.edit

Metrics Summary:  {'pre': {'rewrite_acc': 0.75}, 'post': {'rewrite_acc': 0.0}}
[{'pre': {'rewrite_acc': [1.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}, {'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Joe Biden', 'ground_truth': 'Donald Trump', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}]


In [ ]:
evaluate_chat_template(edited_model,Evaluation_prompts, Evaluation_metrics, device=1)

/mnt/8t/xkw/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/mnt/8t/xkw/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Reliability:  [' Biden:// Biden:// Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden']
Generalization:  [' Biden:// Bidenating Biden Joeating Biden:// Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden']
Locality:  [' Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden']
Portability:  [' Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden Biden']


#### LoRA

In [4]:

from easyeditor import LoRAHyperParams

hparams = LoRAHyperParams.from_hparams('./hparams/LoRA/llama3-8b.yaml')
hparams.device = 1
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,

    sequential_edit=True,
)

2024-11-11 15:40:52,347 - easyeditor.editors.editor - INFO - Instantiating model
11/11/2024 15:40:52 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-11 15:40:55,257 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
11/11/2024 15:40:55 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/2 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


trainable params: 5,112,576 || all params: 8,035,373,888 || trainable%: 0.06362586323002473
Executing LoRA algo for: [Who is the current President of the United States?] -> [Donald Trump]
Epoch: 0
Batch loss 2.6429638862609863
Total loss 2.6429638862609863
Epoch: 1
Batch loss 1.3599387407302856
Total loss 1.3599387407302856
Epoch: 2
Batch loss 0.5418473482131958
Total loss 0.5418473482131958
Epoch: 3
Batch loss 0.5228520035743713
Total loss 0.5228520035743713
Epoch: 4
Batch loss 0.4603128731250763
Total loss 0.4603128731250763
Epoch: 5
Batch loss 0.39001449942588806
Total loss 0.39001449942588806
Epoch: 6
Batch loss 0.37775060534477234
Total loss 0.37775060534477234
Epoch: 7
Batch loss 0.3374292254447937
Total loss 0.3374292254447937
Epoch: 8
Batch loss 0.27289214730262756
Total loss 0.27289214730262756
Epoch: 9
Batch loss 0.24674639105796814
Total loss 0.24674639105796814
Epoch: 10
Batch loss 0.2413826733827591
Total loss 0.2413826733827591
Epoch: 11
Batch loss 0.2197069525718689
Tota

 50%|█████     | 1/2 [00:16<00:16, 16.69s/it]

Total loss 0.03965139761567116
Executing LoRA algo for: [Who is the current President of the United States?] -> [Joe Biden]
Epoch: 0
Batch loss 13.705687522888184
Total loss 13.705687522888184
Epoch: 1
Batch loss 0.8341963291168213
Total loss 0.8341963291168213
Epoch: 2
Batch loss 0.09587246924638748
Total loss 0.09587246924638748
Epoch: 3
Batch loss 0.022295663133263588
Total loss 0.022295663133263588
Epoch: 4
Batch loss 0.0027309246361255646
Total loss 0.0027309246361255646
Epoch: 5
Batch loss 0.0013770213117823005
Total loss 0.0013770213117823005
Epoch: 6
Batch loss 0.0009390695486217737
Total loss 0.0009390695486217737
Epoch: 7
Batch loss 0.0031901171896606684
Total loss 0.0031901171896606684
Epoch: 8
Batch loss 0.00013749384379480034
Total loss 0.00013749384379480034
Epoch: 9
Batch loss 0.00010918414773186669
Total loss 0.00010918414773186669
Epoch: 10
Batch loss 9.506048081675544e-05
Total loss 9.506048081675544e-05
Epoch: 11
Batch loss 0.00010793243563966826
Total loss 0.0001079

100%|██████████| 2/2 [00:30<00:00, 15.31s/it]
2024-11-11 15:41:32,361 - easyeditor.editors.editor - INFO - 0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
11/11/2024 15:41:32 - INFO - easyeditor.editors.editor -   0 editing: Who is the current President of the United States? -> Donald Trump  

 {'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who is the current President of the United States?', 'target_new': 'Donald Trump', 'ground_truth': 'Joe Biden', 'portability': {}, 'locality': {}, 'subject': 'President'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {

Total loss 3.039828698092606e-06
Metrics Summary:  {'pre': {'rewrite_acc': 0.75}, 'post': {'rewrite_acc': 0.5}}


In [6]:
evaluate_chat_template(edited_model, Evaluation_prompts, Evaluation_metrics,device=1)

Reliability:  Joe Biden Biden Joe Biden Joe Biden Joe Biden Joe Biden Biden Joe Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Joe Biden Biden Jo

#### Prompt

In [4]:
from transformers import LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-3-8b-instruct').to('cuda:1')


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
Evaluation_prompts = [ "Who is the current President of the United States?" ,
            'Who is the head of state in the United States right now?',
            "Where is the capital of the United States?" ,
            'Where is the current U.S. President born?']

# edit_prompt = 'The current President of the United States is Joe Biden.'
edit_prompt = 'Information: The U.S. President changed from Biden to Trump, and finally back to Biden again. Based on the information, answer the following questions and dont answer I cant provide information:'

Evaluation_prompts = [ edit_prompt + ' ' + prompt for prompt in Evaluation_prompts]
evaluate_chat_template(model, Evaluation_prompts,Evaluation_metrics, device=1)


Reliability:  Based on the information provided, the current President of the United States is Joe Biden.
Generalization:  Based on the information provided, the head of state in the United States right now is Joe Biden.
Locality:  The capital of the United States is Washington, D.C.
Portability:  Based on the information provided, the current U.S. President is Joe Biden. Joe Biden was born in Scranton, Pennsylvania, and later moved to Wilmington, Delaware, where he grew up.
